# Linear Optics Phase Calculation for the BELLA Laser Experimant

## Boaz Nash, _RadiaSoft LLC_

### _LaTeX_ macros

$$
\newcommand{\lambdabar}{{\mkern0.75mu\mathchar '26\mkern -9.75mu\lambda}}
$$

### _Preamble_

In [11]:
import numpy as np

The seed laser passes through a crystal of length L ($L_c$=1cm). In the case that the crystal isn't pumped first,
it arrives at a waist at 50 cm from the end of the crystal. Now, assume it is pumped and heated, giving rise to
a thermal lensing effect. We want to find the ABCD matrix that gets us between these two cases.
We need to go backwards through the unheated crystal and then forward through the heated crystal.
The matrix to pass through the crystal with $n_2 = 0$ is 
$$M_c = \pmatrix{1 & \frac{L_c}{n_0} \cr 0 & 1} $$
The crystal matrix with n_0 and n_2 is [Siegman, p 586]
\begin{equation}
  \bar M_c = \pmatrix{\cos(\gamma L) & \frac{1}{n_0 \gamma} \sin(\gamma L)\cr 
            -(n_0 \gamma) \sin(\gamma L) & \cos(\gamma L) }
\end{equation}

This acts on a ray via
$$\pmatrix{ x_f \cr n_f \theta_f } = \pmatrix{ A & B \cr C & D }\pmatrix{x_i \cr n_i \theta_i}$$
Next, we consider the drift from the crystal to the wavefront sensor of length $L_d$ = 0.5m, the matrix of which is

$$M_d = \pmatrix{1 & L_d \cr 0 & 1} $$

Finally, we need to consider the impact of the curvature of the front and back of the crystal due to heating. This
introduces a thin lens which is represented by the matrix

$$M_{tl} = \pmatrix{1 & 0\cr -\frac{1}{f} & 1} $$

The total matrix we want to apply to transform between the two measurmed wavefronts is
$$M_T = M_d M_{tl} \bar M_c M_c^{-1} M_{tl}^{-1} M_d^{-1}$$

The index of refraction is given by $$n(r) = n_0 - \frac{1}{2} n_2 r^2$$ 
and $\gamma$ is defined as 
$$\gamma = \sqrt{\frac{n_2}{n_0}}$$

Lengths are given in m and angles in radians.

### Calculations

In [49]:
n0 = 1.76
#n2=1.6e-3 #cm^-2
#n2 = 16 #m^-2 from Fenics calculation
n2 = 15.38

flen_front = 4.9 #measured front of crystal focal length from thermal bulging
flen_back = 8.8 #measured back of crystal focal length from thermal bulging
f = flen_back

Lc = 0.025
#Ld = 0.5
Ld = 0.0
Matcinv = np.array( [[ 1, -Lc/n0], [0, 1]])
gamma = np.sqrt(n2/n0)
A = np.cos(gamma*Lc)
B = 1/(n0 * gamma) * np.sin(gamma*Lc)
C = -n0 * gamma * np.sin(gamma * Lc)
D = np.cos(gamma*Lc)
Matcbar = np.array( [[ A, B], [C, D]] )
Matd = np.array( [[ 1, Ld], [0, 1]] )
Matdinv = np.array( [[ 1, -Ld], [0, 1]] )
Mat_tl = np.array( [[ 1, 0], [-1/f, 1]] )
Mat_tlinv = np.array( [[ 1, 0], [1/f, 1]] )
Mattot0 = np.matmul(np.matmul(np.matmul(Matd,Matcbar),Matcinv),Matdinv)
#Mattot1 = np.matmul(np.matmul(np.matmul(np.matmul(np.matmul(Matd,Mat_tl),Matcbar),Matcinv),Mat_tlinv),Matdinv)
Mattot1 = np.matmul(np.matmul(np.matmul(np.matmul(np.matmul(Matd,Mat_tl),Matcbar),Matcinv),Mat_tl),Matdinv)
print(gamma)
print(Mattot0)
print(Mattot1)
Mattot = Mattot1

2.9561184623821086
[[ 9.97270419e-01  2.58459534e-05]
 [-3.84150095e-01  1.00272710e+00]]
[[ 9.97267482e-01  2.58459534e-05]
 [-6.11422206e-01  1.00272416e+00]]


In [38]:
Mat_tl

array([[ 1.        ,  0.        ],
       [-0.11363636,  1.        ]])

In [23]:
-1/Matcbar[1,0]

6.250947070142776

In [45]:
1/(0.025*2.6)

15.384615384615383

Now we construct the initial moment matrix $\Sigma_i$ and transform it via $M$, using the fact that
$$\Sigma_f = M\Sigma_i M^T$$
where $\Sigma_{i,f}$ are the initial and final second moment matrices respectively.

In [60]:
#sigmax = 64e-6 #rms beam size [m]
#sigmax = 1e-3
w0=1.3e-3 #beamsize at WFS in [m] from the fit in data02
#sigma = 0.5 w0
sigmax = 0.5*w0
lambda_rad = 799.89e-9 #wavelength [m]
Zr = np.pi*w0**2/lambda_rad
sigtheta = lambda_rad/(4*np.pi*sigmax) #rms divergence [rad]

sigmai = np.array( [[sigmax**2, 0], [0, sigtheta**2] ] ) 
sigmaf = np.matmul(np.matmul(Mattot, sigmai),np.transpose(Mattot))
print("Rayleigh range[m]",Zr)
print("sigma_theta[rad]",sigtheta)
print("Sigma matrix without pumping:",sigmai)
print("sigmaf:",sigmaf)

Rayleigh range[m] 6.637527140690282
sigma_theta[rad] 9.792803648444322e-05
Sigma matrix without pumping: [[4.22500000e-07 0.00000000e+00]
 [0.00000000e+00 9.58990033e-09]]
sigmaf: [[ 4.20194177e-07 -2.57619753e-07]
 [-2.57619753e-07  1.67588401e-07]]


Now, we relate this to the phase of a Gaussian wavefront.
We consider a coherent Gaussian given by
$$
  E(x) = E_0 e^{-\frac{a+i b}{2} x^2}
$$
We define a second moment matrix via
$$
  \Sigma = \pmatrix{\Sigma_{xx} & \Sigma_{x\theta} \cr \Sigma_{x\theta} & \Sigma_{\theta\theta}}
$$
We can relate the second moment matrix to the real parameters $a$ and $b$.
\begin{align}
  \Sigma_{xx}           &= \frac{1}{2a} \\
  \Sigma_{x\theta}      &= \frac{\lambda}{4\pi} \frac{b}{a} \\
  \Sigma_{\theta\theta} &= \frac{\lambda}{4\pi} \left( a+ \frac{b^2}{a}\right)
\end{align}

From the equations for $\Sigma_{xx}$ and $\Sigma_{x\theta}$, we find
$$ b = \frac{2\pi}{\lambda} \frac{\Sigma_{x\theta}}{\Sigma_{xx}} $$

Now, we want to compare to the phase measurement from the wavefront sensor.
The measured phase is given by
$$
  \Phi(x) = \frac{\lambda}{2\pi} \phi(x) = \frac{\lambda}{2\pi} \left(\phi_0 - \frac{b}{2}x^2 \right),
$$
Where $\Phi$ denotes the phase expressed as a length.

Comparing this to the definition of the phase curvature by JVT, we find
$$
  \alpha = -\frac{\lambda b}{4\pi} = -\frac{1}{2}\frac{\sigma_{x\theta}}{\sigma_{xx}}
$$

In [63]:
alpha_micron_m2_JVT = 6.5e-7 #This number from JVT email
alpha_meter_m2_JVT = 1e12*lambda_rad*alpha_micron_m2
lambda_micron = 0.8
alpha = -0.5* sigmaf[0,1]/sigmaf[0,0]
print(alpha, alpha_meter_m2_JVT)

0.30654845737224323 0.5199285


Now do the calculation using the <a href=https://en.wikipedia.org/wiki/Gaussian_beam#Complex_beam_parameter>complex beam parameter</a>, $q$.
$$u(r,z)=\frac{1}{q} e^{-ik \frac{r^2}{2q}}$$
$q$ transforms under an ABCD matrix via
$$q_f = \frac{A q_i + B}{C q_i + D}$$
We connect to the approach above with
$$a+ib = -\frac{ik}{q}$$

In [ ]:
lam